# Weather API Client Demos
Use this notebook to verify the local weather API wrappers for Tomorrow.io, Open-Meteo, Visual Crossing, NOAA CDO, WeatherAPI.com, OpenWeather, and Weatherbit.


In [16]:
import json
from pathlib import Path
import sys
PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
CONFIG_PATH = Path('../weather_config.json')
CONFIG = json.loads(CONFIG_PATH.read_text())
LOCATIONS = CONFIG.get('locations', {})
LOCATION_ITEMS = []
for name, coords in LOCATIONS.items():
    try:
        lat = float(coords['lat'])
        lon = float(coords['lon'])
    except (KeyError, TypeError, ValueError) as exc:
        raise ValueError(f"Invalid coordinates for location '{name}'. Provide numeric 'lat' and 'lon'.") from exc
    LOCATION_ITEMS.append((name, lat, lon))
if not LOCATION_ITEMS:
    raise ValueError("Define at least one location under 'locations' in weather_config.json.")
def iter_locations():
    for item in LOCATION_ITEMS:
        yield item
LOCATION_ITEMS

[('new_york_ny', 40.78, -73.97),
 ('austin_tx', 30.18, -97.68),
 ('chicago_midway_il', 41.78, -87.76),
 ('los_angeles_ca', 33.94, -118.39),
 ('miami_fl', 25.79, -80.32)]

In [22]:
import datetime as dt
import json
import logging
import threading
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Sequence, Tuple, Union

import matplotlib
matplotlib.use("Agg")

import matplotlib.pyplot as plt  # noqa: E402  (import after backend selection)
import numpy as np  # noqa: E402
import pandas as pd  # noqa: E402

from clients.copernicus_cds_client import CopernicusCdsClient
from clients.iem_asos_client import IemAsosClient
from clients.meteostat_client import MeteostatClient
from clients.nasa_power_client import NasaPowerClient
from clients.noaa_access_client import NoaaIsdClient, NoaaLcdClient
from clients.open_meteo_client import OpenMeteoClient
from clients.openweather_client import OpenWeatherClient
from clients.tomorrow_io_client import TomorrowIOClient
from clients.visual_crossing_client import VisualCrossingClient
from clients.weatherapi_com_client import WeatherApiClient
from clients.weatherbit_client import WeatherbitClient

## Tomorrow.io client
The client reads API keys and defaults from `weather_config.json`. Update the config file before running the calls below.

In [2]:
from clients.tomorrow_io_client import TomorrowIOClient

client = TomorrowIOClient(config_path="../weather_config.json")
client.base_url


'https://api.tomorrow.io/v4'

## Forecast example
Requests the next 6 hours of weather data for Boston, MA. Adjust the location or fields for your use case.

In [3]:
forecast_results = {}
for name, lat, lon in iter_locations():
    request = {
        "location": f"{lat},{lon}",
        "timeframe_hours": 6,
        "fields": ["temperature", "humidity", "windSpeed"],
        "timesteps": ["1h"],
        "units": "metric",
        "timezone": "UTC",
    }
    forecast_results[name] = {
        "single": client.get_forecast(**request),
        "batch": client.get_forecast_batch([request])[0],
    }
forecast_results


ApiError: Tomorrow.io error 429: {"code":429001,"type":"Too Many Calls","message":"The request limit for this resource has been reached for the current rate limit window. Wait and retry the operation, or examine your API request volume."}

## Historical example
Fetches the last 24 hours leading up to the current time. `start` and `end` must be in ISO-8601.

In [ ]:
import datetime as dt

historical_results = {}
end_time = dt.datetime.now(dt.timezone.utc).replace(microsecond=0)
start_time = end_time - dt.timedelta(hours=24)

for name, lat, lon in iter_locations():
    request = {
        "location": f"{lat},{lon}",
        "start_time": start_time,
        "end_time": end_time,
        "fields": ["temperature", "humidity", "windSpeed"],
        "timesteps": ["1h"],
        "units": "metric",
        "timezone": "UTC",
    }
    historical_results[name] = {
        "single": client.get_historical(**request),
        "batch": client.get_historical_batch([request])[0],
    }
historical_results


## Next steps
Explore the payloads, convert them to pandas DataFrames, or export to other formats as needed.

## Open-Meteo client
Open-Meteo does not require an API key. Defaults (timezone, hourly variables, units) come from `weather_config.json`.

In [4]:
from clients.open_meteo_client import OpenMeteoClient

open_meteo = OpenMeteoClient(config_path="../weather_config.json")
open_meteo.forecast_url


'https://api.open-meteo.com/v1/forecast'

## Open-Meteo forecast example
Requests the upcoming 2 days of data for Boston, MA. Adjust location, variables, or duration to suit your needs.

In [5]:
open_meteo_forecasts = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
        "daily": ["temperature_2m_max", "temperature_2m_min"],
        "forecast_days": 2,
    }
    open_meteo_forecasts[name] = {
        "single": open_meteo.get_forecast(**request),
        "batch": open_meteo.get_forecast_batch([request])[0],
    }
open_meteo_forecasts


{'new_york_ny': {'single': {'latitude': 40.78858,
   'longitude': -73.96611,
   'generationtime_ms': 0.11396408081054688,
   'utc_offset_seconds': 0,
   'timezone': 'GMT',
   'timezone_abbreviation': 'GMT',
   'elevation': 40.0,
   'hourly_units': {'time': 'iso8601',
    'temperature_2m': '°C',
    'relative_humidity_2m': '%',
    'wind_speed_10m': 'km/h'},
   'hourly': {'time': ['2025-11-13T00:00',
     '2025-11-13T01:00',
     '2025-11-13T02:00',
     '2025-11-13T03:00',
     '2025-11-13T04:00',
     '2025-11-13T05:00',
     '2025-11-13T06:00',
     '2025-11-13T07:00',
     '2025-11-13T08:00',
     '2025-11-13T09:00',
     '2025-11-13T10:00',
     '2025-11-13T11:00',
     '2025-11-13T12:00',
     '2025-11-13T13:00',
     '2025-11-13T14:00',
     '2025-11-13T15:00',
     '2025-11-13T16:00',
     '2025-11-13T17:00',
     '2025-11-13T18:00',
     '2025-11-13T19:00',
     '2025-11-13T20:00',
     '2025-11-13T21:00',
     '2025-11-13T22:00',
     '2025-11-13T23:00',
     '2025-11-14T00:00

## Open-Meteo historical example
Fetches the previous 5 days of archive data for the same location. Dates must be provided in ISO format.

In [6]:
import datetime as dt
historical_end = dt.date.today()
historical_start = historical_end - dt.timedelta(days=5)

open_meteo_historical = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "start_date": historical_start,
        "end_date": historical_end,
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"],
        "daily": ["temperature_2m_max", "temperature_2m_min"],
    }
    open_meteo_historical[name] = {
        "single": open_meteo.get_historical(**request),
        "batch": open_meteo.get_historical_batch([request])[0],
    }
open_meteo_historical


{'new_york_ny': {'single': {'latitude': 40.808434,
   'longitude': -74.0199,
   'generationtime_ms': 5.565881729125977,
   'utc_offset_seconds': 0,
   'timezone': 'GMT',
   'timezone_abbreviation': 'GMT',
   'elevation': 40.0,
   'hourly_units': {'time': 'iso8601',
    'temperature_2m': '°C',
    'relative_humidity_2m': '%',
    'wind_speed_10m': 'km/h'},
   'hourly': {'time': ['2025-11-08T00:00',
     '2025-11-08T01:00',
     '2025-11-08T02:00',
     '2025-11-08T03:00',
     '2025-11-08T04:00',
     '2025-11-08T05:00',
     '2025-11-08T06:00',
     '2025-11-08T07:00',
     '2025-11-08T08:00',
     '2025-11-08T09:00',
     '2025-11-08T10:00',
     '2025-11-08T11:00',
     '2025-11-08T12:00',
     '2025-11-08T13:00',
     '2025-11-08T14:00',
     '2025-11-08T15:00',
     '2025-11-08T16:00',
     '2025-11-08T17:00',
     '2025-11-08T18:00',
     '2025-11-08T19:00',
     '2025-11-08T20:00',
     '2025-11-08T21:00',
     '2025-11-08T22:00',
     '2025-11-08T23:00',
     '2025-11-09T00:00',

## Visual Crossing client
Visual Crossing uses the Timeline Weather API and requires an API key provided in `weather_config.json`.

In [7]:
from clients.visual_crossing_client import VisualCrossingClient

visual_crossing = VisualCrossingClient(config_path="../weather_config.json")
visual_crossing.base_url


'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline'

## Visual Crossing forecast example
Retrieves upcoming conditions for the configured coordinates, including current, daily, and hourly data segments.

In [8]:
visual_crossing_forecasts = {}
# Limit to first location to reduce API cost/quota
first = next(iter_locations())
name, lat, lon = first
request = {
    "location": f"{lat},{lon}",
    "start": None,
    "end": None,
    "include": ["hours"],
    "unit_group": "metric",
}
try:
    single = visual_crossing.get_forecast(**request)
except Exception as e:
    single = {"error": str(e)}
try:
    batch = visual_crossing.get_forecast_batch([request])[0]
except Exception as e:
    batch = {"error": str(e)}
visual_crossing_forecasts[name] = {"single": single, "batch": batch}
visual_crossing_forecasts


{'new_york_ny': {'single': {'queryCost': 1,
   'latitude': 40.78,
   'longitude': -73.97,
   'resolvedAddress': '40.78,-73.97',
   'address': '40.78,-73.97',
   'timezone': 'America/New_York',
   'tzoffset': -5.0,
   'days': [{'datetime': '2025-11-12',
     'datetimeEpoch': 1762923600,
     'tempmax': 9.4,
     'tempmin': 3.4,
     'temp': 6.6,
     'feelslikemax': 8.7,
     'feelslikemin': -1.1,
     'feelslike': 4.3,
     'dew': -4.3,
     'humidity': 45.9,
     'precip': 0.0,
     'precipprob': 2.0,
     'precipcover': 0.0,
     'preciptype': None,
     'snow': 0.0,
     'snowdepth': 0.0,
     'windgust': 51.8,
     'windspeed': 26.0,
     'winddir': 246.5,
     'pressure': 1009.2,
     'cloudcover': 83.0,
     'visibility': 16.0,
     'solarradiation': 73.7,
     'solarenergy': 6.3,
     'uvindex': 4.0,
     'severerisk': 10.0,
     'sunrise': '06:39:45',
     'sunriseEpoch': 1762947585,
     'sunset': '16:40:02',
     'sunsetEpoch': 1762983602,
     'moonphase': 0.75,
     'condit

## Visual Crossing historical example
Queries the previous 7 days using the timeline endpoint with explicit start and end dates.

In [9]:
visual_crossing_historical = {}
# Limit to first location and 1-day window to reduce API cost/quota
import datetime as dt
first = next(iter_locations())
name, lat, lon = first
start = dt.date.today() - dt.timedelta(days=2)
end = start
request = {
    "location": f"{lat},{lon}",
    "start": start,
    "end": end,
    "include": ["hours"],
    "unit_group": "metric",
}
try:
    single = visual_crossing.get_historical(**request)
except Exception as e:
    single = {"error": str(e)}
try:
    batch = visual_crossing.get_historical_batch([request])[0]
except Exception as e:
    batch = {"error": str(e)}
visual_crossing_historical[name] = {"single": single, "batch": batch}
visual_crossing_historical


{'new_york_ny': {'single': {'queryCost': 24,
   'latitude': 40.78,
   'longitude': -73.97,
   'resolvedAddress': '40.78,-73.97',
   'address': '40.78,-73.97',
   'timezone': 'America/New_York',
   'tzoffset': -5.0,
   'days': [{'datetime': '2025-11-11',
     'datetimeEpoch': 1762837200,
     'tempmax': 5.0,
     'tempmin': 0.7,
     'temp': 2.9,
     'feelslikemax': 2.0,
     'feelslikemin': -4.9,
     'feelslike': -1.2,
     'dew': -6.4,
     'humidity': 50.4,
     'precip': 0.0,
     'precipprob': 0.0,
     'precipcover': 0.0,
     'preciptype': None,
     'snow': 0.0,
     'snowdepth': 0.0,
     'windgust': 56.8,
     'windspeed': 35.4,
     'winddir': 273.4,
     'pressure': 1007.2,
     'cloudcover': 85.2,
     'visibility': 16.0,
     'solarradiation': 75.4,
     'solarenergy': 6.5,
     'uvindex': 6.0,
     'severerisk': 10.0,
     'sunrise': '06:38:33',
     'sunriseEpoch': 1762861113,
     'sunset': '16:40:57',
     'sunsetEpoch': 1762897257,
     'moonphase': 0.71,
     'cond

## NOAA ISD client
Integrated Surface Database (global-hourly) via the Access Data Service.


In [18]:
noaa_isd = NoaaIsdClient(config_path="../weather_config.json")
noaa_isd.base_url, noaa_isd.dataset


('https://www.ncei.noaa.gov/access/services/data/v1', 'global-hourly')

## NOAA ISD observation example
Fetch the past 24 hours of hourly observations for each configured location.


In [23]:
import datetime as dt

DEMO_NOAA_ISD_START = dt.datetime(2024, 3, 1, tzinfo=dt.timezone.utc)
DEMO_NOAA_ISD_END = DEMO_NOAA_ISD_START + dt.timedelta(days=1)

noaa_isd_results = {}
for name, *_coords in iter_locations():
    station_id = LOCATIONS.get(name, {}).get('noaaIsdStation')
    if not station_id:
        print(f"{name}: missing 'noaaIsdStation'; skipping.")
        continue
    payload = noaa_isd.get_observations(
        station_id=station_id,
        start_time=DEMO_NOAA_ISD_START,
        end_time=DEMO_NOAA_ISD_END,
    )
    noaa_isd_results[name] = payload
    print(f"{name}: {len(payload)} ISD rows for {DEMO_NOAA_ISD_START.date()}")

sample_isd_city, sample_isd_payload = next(iter(noaa_isd_results.items()))
pd.DataFrame(sample_isd_payload)[['DATE', 'TMP', 'DEW', 'WND']].head()


new_york_ny: 26 ISD rows for 2024-03-01
austin_tx: 47 ISD rows for 2024-03-01
chicago_midway_il: 30 ISD rows for 2024-03-01
los_angeles_ca: 41 ISD rows for 2024-03-01
miami_fl: 36 ISD rows for 2024-03-01


,DATE,TMP,DEW,WND
0,2024-03-01T00:51:00,"+0033,5","-0100,5","290,5,N,0021,5"
1,2024-03-01T01:51:00,"+0022,5","-0117,5","290,5,N,0026,5"
2,2024-03-01T02:51:00,"+0017,5","-0117,5","999,9,V,0015,5"
3,2024-03-01T03:51:00,"+0017,5","-0117,5","300,5,N,0036,5"
4,2024-03-01T04:51:00,"+0011,5","-0117,5","999,9,V,0021,5"


## NOAA LCD observation example
Hourly Local Climatological Data via the same access service.


In [25]:
import datetime as dt

DEMO_NOAA_LCD_START = dt.datetime(2024, 3, 1, tzinfo=dt.timezone.utc)
DEMO_NOAA_LCD_END = DEMO_NOAA_LCD_START + dt.timedelta(days=1)

noaa_lcd = NoaaLcdClient(config_path="../weather_config.json")
noaa_lcd_results = {}
for name, *_coords in iter_locations():
    station_id = LOCATIONS.get(name, {}).get('noaaLcdStation')
    if not station_id:
        print(f"{name}: missing 'noaaLcdStation'; skipping.")
        continue
    payload = noaa_lcd.get_observations(
        station_id=station_id,
        start_time=DEMO_NOAA_LCD_START,
        end_time=DEMO_NOAA_LCD_END,
    )
    noaa_lcd_results[name] = payload
    print(f"{name}: {len(payload)} LCD rows for {DEMO_NOAA_LCD_START.date()}")


new_york_ny: 25 LCD rows for 2024-03-01
austin_tx: 41 LCD rows for 2024-03-01
chicago_midway_il: 29 LCD rows for 2024-03-01


ApiError: NOAA access error 503: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>503 Service Unavailable</title>
</head><body>
<h1>Service Unavailable</h1>
<p>The server is temporarily unable to service your
request due to maintenance downtime or capacity
problems. Please try again later.</p>
<p>Additionally, a 503 Service Unavailable
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>


In [ ]:
sample_lcd_city, sample_lcd_payload = next(iter(noaa_lcd_results.items()))
pd.DataFrame(sample_lcd_payload)[["DATE", "HourlyDryBulbTemperature", "HourlyDewPointTemperature"]].head()


NameError: name 'noaa_lcd_results' is not defined

## Meteostat client
Hourly observations via the Meteostat Python library (Point API).


In [26]:
meteostat_client = MeteostatClient(config_path="../weather_config.json")


In [27]:
import datetime as dt
METEOSTAT_DEMO_START = dt.datetime(2024, 3, 1)
METEOSTAT_DEMO_END = METEOSTAT_DEMO_START + dt.timedelta(days=1)
meteostat_results = {}
for name, lat, lon in iter_locations():
    payload = meteostat_client.get_hourly(
        location=(lat, lon),
        start_time=METEOSTAT_DEMO_START,
        end_time=METEOSTAT_DEMO_END,
    )
    meteostat_results[name] = payload
    print(f"{name}: {len(payload)} Meteostat rows for {METEOSTAT_DEMO_START.date()}")
sample_meteostat_city, sample_meteostat_payload = next(iter(meteostat_results.items()))
pd.DataFrame(sample_meteostat_payload)[['timestamp', 'temp', 'dwpt']].head()


new_york_ny: 25 Meteostat rows for 2024-03-01
austin_tx: 25 Meteostat rows for 2024-03-01
chicago_midway_il: 25 Meteostat rows for 2024-03-01
los_angeles_ca: 25 Meteostat rows for 2024-03-01
miami_fl: 25 Meteostat rows for 2024-03-01


,timestamp,temp,dwpt
0,2024-03-01 00:00:00,3.0,-10.3
1,2024-03-01 01:00:00,3.3,-9.3
2,2024-03-01 02:00:00,3.3,-10.0
3,2024-03-01 03:00:00,2.0,-11.5
4,2024-03-01 04:00:00,1.0,-11.7


## NASA POWER client
NASA POWER hourly API via REST.


In [28]:
nasa_power = NasaPowerClient(config_path="../weather_config.json")


In [29]:
import datetime as dt

NASA_DEMO_START = dt.datetime(2024, 3, 1)
NASA_DEMO_END = NASA_DEMO_START + dt.timedelta(days=1)

nasa_power_results = {}
for name, lat, lon in iter_locations():
    payload = nasa_power.get_hourly(
        location=(lat, lon),
        start_time=NASA_DEMO_START,
        end_time=NASA_DEMO_END,
    )
    nasa_power_results[name] = payload
    count = 0
    props = payload.get('properties', {}) if isinstance(payload, dict) else {}
    param_block = props.get('parameter', {})
    if param_block:
        first_param = next(iter(param_block.values()))
        count = len(first_param)
    print(f"{name}: {count} NASA POWER rows for {NASA_DEMO_START.date()}")

sample_nasa_city, sample_nasa_payload = next(iter(nasa_power_results.items()))
parameters = sample_nasa_payload.get('properties', {}).get('parameter', {})
rows = {}
for param, series in parameters.items():
    for stamp, value in series.items():
        rows.setdefault(stamp, {})[param] = value
records = []
for stamp, values in rows.items():
    ts = dt.datetime.strptime(stamp, '%Y%m%d%H')
    values = dict(values)
    values['timestamp'] = ts
    records.append(values)
pd.DataFrame(records)[['timestamp', 'T2M', 'RH2M', 'WS10M']].head()


new_york_ny: 48 NASA POWER rows for 2024-03-01
austin_tx: 48 NASA POWER rows for 2024-03-01
chicago_midway_il: 48 NASA POWER rows for 2024-03-01
los_angeles_ca: 48 NASA POWER rows for 2024-03-01
miami_fl: 48 NASA POWER rows for 2024-03-01


,timestamp,T2M,RH2M,WS10M
0,2024-03-01 00:00:00,-2.25,82.05,3.62
1,2024-03-01 01:00:00,-2.98,85.25,3.30
2,2024-03-01 02:00:00,-3.44,87.24,3.04
3,2024-03-01 03:00:00,-3.62,87.88,2.84
4,2024-03-01 04:00:00,-3.78,89.09,2.62


## Copernicus ERA5 datasets
Four CDS-backed providers are configured: ERA5 single levels, ERA5-Land, ERA5 pressure levels, and ERA5-Land time-series. Each provider uses its own bounding box metadata in `weather_config.json`. The cell below loops through all four providers and fetches a one-day sample for every configured location.

In [30]:
from clients.noaa_access_client import NoaaIsdClient, NoaaLcdClient
from clients.meteostat_client import MeteostatClient
from clients.nasa_power_client import NasaPowerClient
from clients.iem_asos_client import IemAsosClient
from clients.copernicus_cds_client import CopernicusCdsClient

from IPython.display import display
import datetime as dt

COPERNICUS_DEMOS = [
    {
        "label": "ERA5 single levels",
        "provider": "copernicus_era5_single",
        "area_attr": "copernicusEra5Area",
        "date": dt.date(2024, 1, 1),
    },
    {
        "label": "ERA5-Land",
        "provider": "copernicus_era5_land",
        "area_attr": "copernicusEra5LandArea",
        "date": dt.date(2024, 1, 1),
    },
    {
        "label": "ERA5 pressure levels",
        "provider": "copernicus_era5_pressure",
        "area_attr": "copernicusEra5Area",
        "date": dt.date(2024, 1, 1),
    },
    {
        "label": "ERA5-Land time-series",
        "provider": "copernicus_era5_land_timeseries",
        "area_attr": None,
        "date": dt.date(2024, 1, 1),
    },
]


def _run_copernicus_demo(cfg):
    client = CopernicusCdsClient(config_path="../weather_config.json", provider=cfg["provider"])
    results = {}
    for name, lat, lon in iter_locations():
        extras = LOCATIONS.get(name, {})
        area_attr = cfg.get("area_attr")
        area = extras.get(area_attr) if area_attr else None
        if area_attr and not area:
            print(f"{cfg['label']} -> {name}: missing {area_attr}; skipping.")
            continue
        payload = client.get_dataset(
            area=area,
            start_date=cfg["date"],
            end_date=cfg["date"],
            latitude=lat,
            longitude=lon,
        )
        results[name] = payload
        length = 0 if payload is None else len(payload)
        print(f"{cfg['label']} -> {name}: {length} rows on {cfg['date']}")
    return results


def _preview_dataframe(df):
    cols = [col for col in ("timestamp", "pressure_level", "t2m", "tp", "temperature", "u10", "v10") if col in df.columns]
    if not cols:
        cols = list(df.columns[:5])
    return df[cols].head()

copernicus_results = {}
for cfg in COPERNICUS_DEMOS:
    print(f"=== {cfg['label']} ===")
    copernicus_results[cfg["provider"]] = _run_copernicus_demo(cfg)

for provider_key, results in copernicus_results.items():
    sample_item = next(((city, payload) for city, payload in results.items() if payload is not None and not payload.empty), None)
    if sample_item:
        city, payload = sample_item
        print(f"Sample preview for {provider_key} ({city})")
        display(_preview_dataframe(payload))
    else:
        print(f"No sample data to preview for {provider_key}.")


=== ERA5 single levels ===


ERA5 single levels -> new_york_ny: 24 rows on 2024-01-01


ERA5 single levels -> austin_tx: 24 rows on 2024-01-01


HTTPError: 500 Server Error: Internal Server Error for url: https://cds.climate.copernicus.eu/api/retrieve/v1/jobs/466fc410-bce4-44a8-8ce2-0dee350b1748?log=True&request=True

## IEM ASOS client
Iowa Environmental Mesonet 1-minute ASOS data.


In [31]:
iem_asos = IemAsosClient(config_path="../weather_config.json")


In [32]:
import datetime as dt

IEM_DEMO_START = dt.datetime(2025, 1, 1)
IEM_DEMO_END = IEM_DEMO_START + dt.timedelta(days=1)

iem_results = {}
for name, *_coords in iter_locations():
    station = LOCATIONS.get(name, {}).get('iemStation')
    network = LOCATIONS.get(name, {}).get('iemNetwork')
    if not station or not network:
        print(f"{name}: missing IEM metadata; skipping.")
        continue
    payload = iem_asos.get_observations(
        station=station,
        network=network,
        start_time=IEM_DEMO_START,
        end_time=IEM_DEMO_END,
    )
    iem_results[name] = payload
    length = 0 if payload is None else len(payload)
    print(f"{name}: {length} IEM rows for {IEM_DEMO_START.date()}")

sample_item = next(((city, payload) for city, payload in iem_results.items() if payload is not None and not payload.empty), None)
if sample_item:
    sample_city, sample_payload = sample_item
    pd.DataFrame(sample_payload)[['timestamp', 'tmpf', 'dwpf', 'sknt']].head()
else:
    print('No IEM sample data available.')


new_york_ny: 1130 IEM rows for 2025-01-01
austin_tx: 937 IEM rows for 2025-01-01
chicago_midway_il: 1249 IEM rows for 2025-01-01
los_angeles_ca: 960 IEM rows for 2025-01-01
miami_fl: 1270 IEM rows for 2025-01-01


## WeatherAPI.com client
WeatherAPI.com uses API key authentication via query parameters; the key lives in `weather_config.json`.

In [33]:
from clients.weatherapi_com_client import WeatherApiClient

weatherapi = WeatherApiClient(config_path="../weather_config.json")
weatherapi.base_url


'https://api.weatherapi.com/v1'

## WeatherAPI.com forecast example
Requests a 3-day forecast (includes hourly segments) for the configured coordinates.

In [34]:
weatherapi_forecasts = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "days": 3,
        "aqi": 'yes',
        "alerts": 'yes',
    }
    weatherapi_forecasts[name] = {
        "single": weatherapi.get_forecast(**request),
        "batch": weatherapi.get_forecast_batch([request])[0],
    }
weatherapi_forecasts


ApiError: WeatherAPI.com returned a non-JSON response.

## WeatherAPI.com historical example
Fetches hourly history for a single day (WeatherAPI supports one date per call on standard plans).

In [35]:
import datetime as dt
history_date = dt.date.today() - dt.timedelta(days=1)
weatherapi_history = {}
first = next(iter_locations())
name, lat, lon = first
request = {
    "location": (lat, lon),
    "date": history_date,
    "aqi": 'yes',
}
try:
    single = weatherapi.get_historical(**request)
except Exception as e:
    single = {"error": str(e)}
try:
    batch = weatherapi.get_historical_batch([request])[0]
except Exception as e:
    batch = {"error": str(e)}
weatherapi_history[name] = {"single": single, "batch": batch}
weatherapi_history


{'new_york_ny': {'single': {'error': "get_historical() got an unexpected keyword argument 'aqi'"},
  'batch': TypeError("get_historical() got an unexpected keyword argument 'aqi'")}}

## OpenWeather client
OpenWeather current and historical APIs both rely on latitude/longitude plus an API key stored in `weather_config.json`.


In [36]:
import importlib
from clients import openweather_client as _openweather_module
importlib.reload(_openweather_module)
from clients.openweather_client import OpenWeatherClient

openweather = OpenWeatherClient(config_path="../weather_config.json")
openweather.current_url, openweather.history_url


('https://api.openweathermap.org/data/2.5/weather',
 'https://history.openweathermap.org/data/2.5/history/city')

## OpenWeather current example
Fetches the latest observations for the configured coordinates.


In [37]:
current_results = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "units": 'metric',
        "language": 'en',
    }
    current_results[name] = {
        "single": openweather.get_current(**request),
        "batch": openweather.get_current_batch([request])[0],
    }
current_results


{'new_york_ny': {'single': {'coord': {'lon': -73.97, 'lat': 40.78},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03n'}],
   'base': 'stations',
   'main': {'temp': 9.22,
    'feels_like': 6.03,
    'temp_min': 7.82,
    'temp_max': 9.97,
    'pressure': 1009,
    'humidity': 62,
    'sea_level': 1009,
    'grnd_level': 1007},
   'visibility': 10000,
   'wind': {'speed': 6.69, 'deg': 260, 'gust': 11.32},
   'clouds': {'all': 40},
   'dt': 1763005957,
   'sys': {'type': 1,
    'id': 5141,
    'country': 'US',
    'sunrise': 1762947579,
    'sunset': 1762983638},
   'timezone': -18000,
   'id': 5125771,
   'name': 'Manhattan',
   'cod': 200},
  'batch': {'coord': {'lon': -73.97, 'lat': 40.78},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03n'}],
   'base': 'stations',
   'main': {'temp': 9.22,
    'feels_like': 6.03,
    'temp_min': 7.82,
    'temp_max': 9.97,
    'pressur

## OpenWeather historical example
Requests sub-hourly history for the last 24 hours (data is returned in hourly buckets).


In [38]:
import datetime as dt
ow_end = dt.datetime.now(dt.timezone.utc)
ow_start = ow_end - dt.timedelta(hours=24)
openweather_history = {}
first = next(iter_locations())
name, lat, lon = first
request = {
    "location": (lat, lon),
    "start_time": ow_start,
    "end_time": ow_end,
    "units": 'metric',
    "interval_type": 'hour',
}
try:
    single = openweather.get_historical(**request)
except Exception as e:
    single = {"error": str(e)}
try:
    batch = openweather.get_historical_batch([request])[0]
except Exception as e:
    batch = {"error": str(e)}
openweather_history[name] = {"single": single, "batch": batch}
openweather_history


{'new_york_ny': {'single': {'message': 'Count: 24',
   'cod': '200',
   'city_id': 1,
   'calctime': 0.044515794,
   'cnt': 24,
   'list': [{'dt': 1762920000,
     'main': {'temp': 4.22,
      'feels_like': -1,
      'pressure': 1013,
      'humidity': 58,
      'temp_min': 3.23,
      'temp_max': 4.8},
     'wind': {'speed': 8.49, 'deg': 250},
     'clouds': {'all': 75},
     'weather': [{'id': 803,
       'main': 'Clouds',
       'description': 'broken clouds',
       'icon': '04n'}]},
    {'dt': 1762923600,
     'main': {'temp': 4.15,
      'feels_like': -1.33,
      'pressure': 1013,
      'humidity': 57,
      'temp_min': 3.03,
      'temp_max': 4.8},
     'wind': {'speed': 9.26, 'deg': 250, 'gust': 16.98},
     'clouds': {'all': 75},
     'weather': [{'id': 500,
       'main': 'Rain',
       'description': 'light rain',
       'icon': '10n'}],
     'rain': {'1h': 0.25}},
    {'dt': 1762927200,
     'main': {'temp': 3.73,
      'feels_like': -1.38,
      'pressure': 1013,
      'h

## Weatherbit client
Weatherbit exposes hourly forecast data and sub-hourly historical archives; the API key lives in `weather_config.json`.


In [39]:
from clients.weatherbit_client import WeatherbitClient

weatherbit = WeatherbitClient(config_path="../weather_config.json")
weatherbit.forecast_url


'https://api.weatherbit.io/v2.0/forecast/hourly'

## Weatherbit forecast example
Requests the next 48 hours of hourly forecast data for the configured coordinates.


In [40]:
weatherbit_forecasts = {}
for name, lat, lon in iter_locations():
    request = {
        "location": (lat, lon),
        "hours": 48,
        "units": 'M',
    }
    weatherbit_forecasts[name] = {
        "single": weatherbit.get_forecast(**request),
        "batch": weatherbit.get_forecast_batch([request])[0],
    }
weatherbit_forecasts


ApiError: Weatherbit error 403: {"error":"Your API key does not allow access to this endpoint."}


## Weatherbit historical example
Fetches the previous 24 hours using the sub-hourly historical endpoint (UTC timestamps required).


In [41]:
import datetime as dt
wb_end = dt.datetime.now(dt.timezone.utc)
wb_start = wb_end - dt.timedelta(hours=24)
weatherbit_history = {}
first = next(iter_locations())
name, lat, lon = first
request = {
    "location": (lat, lon),
    "start_time": wb_start,
    "end_time": wb_end,
    "units": 'M',
    "tz": 'UTC',
}
try:
    single = weatherbit.get_historical(**request)
except Exception as e:
    single = {"error": str(e)}
try:
    batch = weatherbit.get_historical_batch([request])[0]
except Exception as e:
    batch = {"error": str(e)}
weatherbit_history[name] = {"single": single, "batch": batch}
weatherbit_history


{'new_york_ny': {'single': {'error': 'Weatherbit error 403: {"error":"Your API Key does not allow access to this endpoint."}\n'},
  'batch': clients.weatherbit_client.ApiError('Weatherbit error 403: {"error":"Your API Key does not allow access to this endpoint."}\n')}}